In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd "/content/gdrive/Shareddrives/Shared/NLP_Final/dataset/"
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/19080pKGbnbJm4nmj1UCKqoDt3kuPBSIk/NLP_Final/dataset
bert_pretrained.mdl  fixed_train.gsheet  NewData/     tweetData.csv
fixed_test.csv       fixed_valid.csv     Origin/
fixed_test.gsheet    fixed_valid.gsheet  Prompt/
fixed_train.csv      LongDialogues/      TextATextB/


## PreProcess Data

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
trainData = pd.read_csv('fixed_train.csv')
validData = pd.read_csv('fixed_valid.csv')
testData = pd.read_csv('fixed_test.csv')

In [ ]:
def lowercase(df):
    df.loc[:, 'prompt'] = df['prompt'].str.lower()
    df.loc[:, 'utterance'] = df['utterance'].str.lower()
    
lowercase(trainData)
lowercase(validData)
lowercase(testData)

In [ ]:
import re, string
def replace_comma(df):
    df.loc[:,'prompt'] = df['prompt'].str.replace('_comma_', ',')
    df.loc[:,'utterance'] = df['utterance'].str.replace('_comma_', ',')

replace_comma(trainData)
replace_comma(validData)
replace_comma(testData)



In [ ]:
# concatenate conversations
def concate_conv(df):
    UTTERANCE = pd.DataFrame(columns=df.columns).drop(['utterance_idx'], axis=1)

    conv_id_groups = df.groupby(['conv_id'])

    for _, indices in tqdm(conv_id_groups.groups.items()):
        conv_rows = df.loc[indices].copy().sort_values(['utterance_idx'])
        conv_row = conv_rows.iloc[0].drop(['utterance_idx'])
        
        text = ' '.join(row['utterance'] for _, row in conv_rows.iterrows())
        conv_row['utterance'] = text
        UTTERANCE = UTTERANCE.append(conv_row, ignore_index=True)
        
    return UTTERANCE

new_train = concate_conv(trainData)
new_valid = concate_conv(validData)
new_test = concate_conv(testData)

  0%|          | 0/19533 [00:00<?, ?it/s]

  0%|          | 0/2770 [00:00<?, ?it/s]

  0%|          | 0/2547 [00:00<?, ?it/s]

In [ ]:
new_train.to_csv('NewData/new_train.csv', index=False, encoding='utf8')
new_valid.to_csv('NewData/new_valid.csv', index=False, encoding='utf8')
new_test.to_csv('NewData/new_test.csv', index=False, encoding='utf8')